In [1]:
import sys
sys.path.append('../src')

import os
import cobra
import pandas as pd
import omg 
from plot_multiomics import *
from tqdm import tqdm

In [2]:
user_params = {
    'host': 'ecoli', # ecoli or ropacus
    'modelfile': '../data/models/iJO1366_MVA.json',
    'cerevisiae_modelfile': '../data/models/iMM904.json', 
    'timestart': 0.0,
    'timestop': 8.0,
    'numtimepoints': 9,
    # TODO: Update designs file & designs file path
    'designsfile': 'ARTrecommendations.csv',
    'designsfilepath': '../data/art_output',
    'mapping_file': '../mapping/inchikey_to_cid.txt',
    'output_file_path': 'data/omg_output',
    'edd_omics_file_path': '../data/omg_output/edd/',
    'numreactions': 8,
    'numinstances': 10,
    'ext_metabolites': {
        'glc__D_e': 22.203,
        'nh4_e': 18.695,
        'pi_e': 69.454,
        'so4_e': 2.0,
        'mg2_e': 2.0,
        'k_e': 21.883,
        'na1_e': 103.7,
        'cl_e': 27.25,
        'isoprenol_e': 0.0,
        'ac_e': 0.0,
        'for_e': 0.0,
        'lac__D_e': 0.0,
        'etoh_e': 0.0
    },
    'initial_OD': 0.01,
    'BIOMASS_REACTION_ID': 'BIOMASS_Ec_iJO1366_core_53p95M'
} 

### 1. Getting and preparing the metabolic model

In [3]:
file_name = user_params['modelfile']
model = cobra.io.load_json_model(file_name)

# Add minimum flux constraints for production of isoprenol and formate and limit oxygen intake
iso = 'EX_isoprenol_e'
iso_cons = model.problem.Constraint(model.reactions.EX_isoprenol_e.flux_expression,lb = 0.20)
model.add_cons_vars(iso_cons)
for_cons = model.problem.Constraint(model.reactions.EX_for_e.flux_expression,lb = 0.10)
model.add_cons_vars(for_cons)
o2_cons = model.problem.Constraint(model.reactions.EX_o2_e.flux_expression,lb = -8.0)
model.add_cons_vars(o2_cons)

# Constrain several central carbon metabolism fluxes to more realistic upper and lower bounds
CC_rxn_names = ['ACCOAC','MDH','PTAr','CS','ACACT1r','PPC','PPCK','PFL']
for reaction in CC_rxn_names:
    reaction_constraint = model.problem.Constraint(model.reactions.get_by_id(reaction).flux_expression,lb = -1.0,ub =  1.0)
    model.add_cons_vars(reaction_constraint)

# Create also a similar model with a higher production of isoprenol, which we will use with MOMA to simulate bioengineered strains:
modelHI = model.copy()
iso_cons = modelHI.problem.Constraint(modelHI.reactions.EX_isoprenol_e.flux_expression,lb = 0.25)
modelHI.add_cons_vars(iso_cons)


Set parameter Username
Academic license - for non-commercial use only - expires 2024-12-30
Read LP format model from file C:\Users\mexis\AppData\Local\Temp\tmpxi9ht02x.lp
Reading time = 0.06 seconds
: 1824 rows, 5194 columns, 20480 nonzeros


### 2. Obtaining times series for the WT

In [ ]:
# Create grid for simulation
t0 = user_params['timestart']
tf = user_params['timestop']
points = user_params['numtimepoints']
tspan, delt = np.linspace(t0, tf, points, dtype='float64', retstep=True)
grid = (tspan, delt)

# Use model to obtain the time series for fluxes, OD and external metabolites
solution_TS, model_TS, cell, Emets, Erxn2Emet = \
    omg.get_flux_time_series(model, user_params['ext_metabolites'], grid, user_params)

# perform the same calculation for the model with higher isoprenol production that we created above
solutionHI_TS, modelHI_TS, cellHI, EmetsHI, Erxn2EmetHI = \
    omg.get_flux_time_series(modelHI, user_params['ext_metabolites'], grid, user_params)


### 3. Getting bioengineered flux profiles through MOMA

In [ ]:
# Read the file with suggested designs (i.e. reactions KOs and Overexpressions)
